In [1]:
import torch
import torchvision
from torch.utils.data import Dataset
import torchvision.transforms.functional as fn
import torchvision.transforms as T
import matplotlib.pyplot as plt
from utilities import createAnnotation
from model_loader import get_new_model
import pandas as pd
from IPython.display import display
from PIL import Image 
import random
import numpy as np

C:\Users\haowa\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\torchvision\io\image.py:13: UserWarning: Failed to load image Python extension: [WinError 127] Die angegebene Prozedur wurde nicht gefunden
  warn(f"Failed to load image Python extension: {e}")


In [2]:
IMAGESROOTDIR = 'NINCO_OOD_classes'

In [3]:
class ImageDataset(Dataset):
    def __init__(self, rootDir):
        self.rootDir = rootDir
        createAnnotation(self.rootDir)
        self.annotation =  pd.read_csv('output.csv')


    def __getitem__(self, index):
        data_path = self.annotation.iloc[index,0]
        image = Image.open(data_path)
        label = self.annotation.iloc[index,1]
        return image, label

    def __len__(self):
        return len(self.annotation)

In [4]:
# instance of class ImageDataset
# contains all 765 images with their respective labels
imageDataset = ImageDataset(rootDir=IMAGESROOTDIR)

In [5]:
# Constants for the size of the images
SIZE = round(224/0.875)

# given an Index returns the transformed Image
# input: Index: int
# return: tuple(PIL Image, label)
def transform(index):
    assert index <= len(imageDataset)
    image, label = imageDataset[index]
    rescaledImage = fn.resize(img=image, size=[SIZE, SIZE], interpolation=T.InterpolationMode.BICUBIC)
    transformedImage = fn.center_crop(img=rescaledImage, output_size=[SIZE,SIZE])
    return transformedImage,label

In [6]:
# objects for tensor transformation
pilToTensor = T.ToTensor()
tensorToPil = T.ToPILImage()

In [21]:
# Class which is used to get the resized images with label
# input: datasetLength: int
# output:{'image': Tensor, 'label': String}
class DataLoader(Dataset):
    def __init__(self, datasetLength):
        self.datasetLength = datasetLength
   
    def __getitem__(self, index):
        assert (0 < index <= self.datasetLength)
        self.index = index
        (picture, label) = transform(index)
        image = pilToTensor(picture)
        #print(image.shape)
        #print(image)
        sample3dim = {'image' : image, 'label' : label}
        image = image.unsqueeze(0)
        #print(image.shape)
        #print(image)
        sample = {'image': image, 'label': label}
        return sample, sample3dim

In [35]:
# Amount of random samples 
BATCHSIZE = 4

dataloader = DataLoader(len(imageDataset))

'''
function creates a random batch of data with a given size
Arguments: batchsize:int
Return: an array with a dict[image:label] 
'''
def createRandomBatch(batchsize):
    assert (0<batchsize <= len(imageDataset))
    batch = []
    batch3dim = []
    indexList = []
    for i in range(batchsize):
        index = random.randint(0,len(imageDataset))
        indexList.append(index)
        sample, sample3dim = dataloader[index]
        batch.append(sample)
        batch3dim.append(sample3dim)
    return batch, batch3dim, indexList

samples, samples3dim, indexList = createRandomBatch(BATCHSIZE)
  
    
    

torch.Size([3, 256, 256])
torch.Size([1, 3, 256, 256])
tensor([[[[0.3529, 0.3686, 0.3922,  ..., 0.6118, 0.5961, 0.5882],
          [0.3765, 0.4235, 0.4353,  ..., 0.6314, 0.6235, 0.6157],
          [0.3608, 0.4039, 0.3922,  ..., 0.6353, 0.6392, 0.6431],
          ...,
          [0.8275, 0.8157, 0.8078,  ..., 0.8000, 0.8118, 0.8275],
          [0.7843, 0.7765, 0.7686,  ..., 0.7922, 0.8039, 0.8235],
          [0.7647, 0.7686, 0.7647,  ..., 0.7882, 0.8000, 0.8196]],

         [[0.2431, 0.2588, 0.2824,  ..., 0.4745, 0.4588, 0.4549],
          [0.2667, 0.3137, 0.3255,  ..., 0.4941, 0.4902, 0.4784],
          [0.2510, 0.2941, 0.2824,  ..., 0.4941, 0.4941, 0.5020],
          ...,
          [0.8275, 0.8157, 0.8078,  ..., 0.5922, 0.6039, 0.6157],
          [0.7843, 0.7765, 0.7686,  ..., 0.5882, 0.5961, 0.6078],
          [0.7647, 0.7686, 0.7647,  ..., 0.5843, 0.5922, 0.6039]],

         [[0.1961, 0.2118, 0.2353,  ..., 0.3412, 0.3333, 0.3255],
          [0.2196, 0.2667, 0.2784,  ..., 0.3608, 0.35

In [36]:
# loads pretrained model
model = get_new_model("convnext_tiny", not_original=True)


'''
function feeds the loaded model with data
Arguments: list[dict[image:tensor,label:str]]
Return: None
'''
def feedModel(samples):
    assert(0<len(samples)<len(imageDataset))
    for sample in samples:
        image, label = sample['image'], sample['label']
        prediction = model(image)
        sample["prediction"]=prediction
        #print(sample["prediction"].shape)
    return samples
        
        
        
newSamples = feedModel(samples)


In [41]:
'''
function extracts the values from the samples dict
Arguments: dict which contains random batch dict
Return: returns the values from samples list
'''
def extractValuesFromDict(samples, key:str):
    values = []
    for dictionary in samples:
        values.append(dictionary[key])
        
    if key == 'label':
        print(values)
    return values


# function to visualize the batch
def visualize(samples):
    tensors = extractValuesFromDict(samples, 'image')
    grid_border_size = 2
    elementsPerRow = 4
    grid = torchvision.utils.make_grid(tensor=tensors, nrow=elementsPerRow, padding=grid_border_size)
    plt.imshow(grid.detach().numpy().transpose((1,2,0)))


plt.figure()
visualize(samples3dim)
extractValuesFromDict(samples3dim, 'label')
plt.axis('off')
plt.ioff()
plt.show()

RuntimeError: The size of tensor a (3) must match the size of tensor b (256) at non-singleton dimension 2